In [ ]:
%uv pip install -U bitsandbytes accelerate
%uv pip install -U transformers diffusers
%uv pip install transformers==4.48.0
%uv pip install peft

In [ ]:
!huggingface-cli login --token "Your_Huggingface_Token_Here"


In [4]:
from diffusers import FluxKontextPipeline
import torch

# Load base model
pipe = FluxKontextPipeline.from_pretrained(
    "black-forest-labs/FLUX.1-Kontext-dev",
    torch_dtype=torch.bfloat16
).to("cuda")
lora_path = "./"  # folder containing the safetensor file
weight_name = "eyeQ1ZtOpNxv-bmOE7iV6_adapter_model.safetensors"

# Load the LoRA weights
pipe.load_lora_weights(
    lora_path,
    weight_name=weight_name,
    adapter_name="my_lora"
)

# Activate the adapter
pipe.set_adapters(["my_lora"])

In [ ]:
!pkill ngrok || true
!ngrok config add-authtoken "add_your_ngrok_token_here"
!pip install fastapi uvicorn pyngrok torch diffusers transformers pillow


In [ ]:
import sys
import io
import base64
import requests
import torch
from fastapi import FastAPI, Form
from fastapi.responses import JSONResponse
from pyngrok import ngrok
from diffusers import FluxKontextPipeline
from diffusers.utils import load_image
from PIL import Image
import uvicorn

# ==============================================================
# 🧩 Initialize model once at startup
# ==============================================================

print("🔧 Loading Flux-Kontext model and LoRA...")

pipe = FluxKontextPipeline.from_pretrained(
    "black-forest-labs/FLUX.1-Kontext-dev",
    torch_dtype=torch.bfloat16
).to("cuda")

# Path to your LoRA adapter
lora_path = "./"  # Folder containing the safetensors file
weight_name = "eyeQ1ZtOpNxv-bmOE7iV6_adapter_model.safetensors"

# Load and activate the adapter
pipe.load_lora_weights(
    lora_path,
    weight_name=weight_name,
    adapter_name="my_lora"
)
pipe.set_adapters(["my_lora"])

print("✅ Model + LoRA loaded successfully.")

# ==============================================================
# 🌐 FastAPI + ngrok setup
# ==============================================================

app = FastAPI()

@app.post("/generate")
async def generate_image(
    image_url: str = Form(...),
    prompt: str = Form(...),
    guidance_scale: float = Form(2.5),
    num_inference_steps: int = Form(30),
    resolution_mode: str = Form("2:3")
):
    """
    Generate an image from a given image_url and prompt using Flux-Kontext + LoRA.
    Returns a Base64-encoded PNG in JSON.
    """
    try:
        print(f"\n📥 New request received:\nPrompt: {prompt}\nImage: {image_url}\nResolution: {resolution_mode}")

        # ----------------------------------------------------------
        # Load input image
        # ----------------------------------------------------------
        input_image = load_image(image_url).convert("RGB")

        # ----------------------------------------------------------
        # Handle resolution / aspect ratio
        # ----------------------------------------------------------
        if resolution_mode == "2:3":
            width, height = 768, 1152
        elif resolution_mode == "3:2":
            width, height = 1152, 768
        elif resolution_mode == "1:1":
            width, height = 1024, 1024
        else:
            width, height = input_image.size  # fallback

        input_image = input_image.resize((width, height), Image.LANCZOS)

        # ----------------------------------------------------------
        # Generate image with LoRA applied
        # ----------------------------------------------------------
        result = pipe(
            image=input_image,
            prompt=prompt,
            guidance_scale=guidance_scale,
            num_inference_steps=num_inference_steps,
            width=width,
            height=height
        ).images[0]

        # ----------------------------------------------------------
        # Convert to Base64 for JSON response
        # ----------------------------------------------------------
        buffered = io.BytesIO()
        result.save(buffered, format="PNG")
        img_base64 = base64.b64encode(buffered.getvalue()).decode("utf-8")

        print("✅ Image generated successfully.")
        return JSONResponse(content={"image_base64": img_base64})

    except Exception as e:
        print(f"❌ Error: {e}", file=sys.stderr)
        return JSONResponse(content={"error": str(e)}, status_code=500)


# ==============================================================
# 🚀 Run the server with ngrok
# ==============================================================

if __name__ == "__main__":
    # Start ngrok tunnel
    public_url = ngrok.connect(8000)
    print(f"\n🚀 Public endpoint: {public_url.public_url}/generate\n")

    # Launch FastAPI server
    uvicorn.run(app, host="0.0.0.0", port=8000)
